[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ManuelEspejo/Machine-Learning-Bases/blob/main/notebooks/03_Aprendizaje-Por-Refuerzo.ipynb) 👈🏻​ **Pulsar para abrir en Colab​**

# ¿Cómo usar estos notebooks?

Si este es el primer notebook que abres en este repositorio, te recomiendo que antes leas el [Manual de uso de estos notebooks](https://github.com/ManuelEspejo/Machine-Learning-Bases/blob/main/docs/manual-notebooks.md) que he creado para que te familiarices con el proyecto y las distintas rutas que puedes seguir, luego puedes volver aquí y continuar.

En este notebook, vamos a profundizar en la **interpretabilidad de los modelos de IA**.

Por otra parte, si aún no has revisado el notebook "[00_Empieza-aquí.ipynb](https://github.com/ManuelEspejo/Machine-Learning-Bases/blob/main/notebooks/00_Empieza-aquí.ipynb)", te sugiero que le eches un vistazo primero para conocer los conceptos básicos. Pero si ya tienes una idea clara de los conceptos básicos y quieres pasar a la práctica, ¡estás en el lugar correcto!

# 04. Interpretabilidad de los Modelos

Cuando entrenamos modelos de Machine Learning, uno de los mayores desafíos no es solo que el modelo funcione bien, sino entender **cómo** y **por qué** toma sus decisiones. Este es el corazón de la interpretabilidad: poder responder a preguntas como *"¿por qué este modelo rechazó mi préstamo?"* o *"¿por qué dice que este tumor es maligno?"*.

## ¿Por qué es importante?

La interpretabilidad de los modelos no es solo un tema técnico, es una cuestión de confianza, responsabilidad y toma de decisiones éticas. Aquí te dejo tres razones principales por las que es crucial:

1. **Confianza en el modelo:** Si no entiendes cómo funciona un modelo, ¿cómo puedes confiar en él? Esto es especialmente importante en áreas críticas como la salud, las finanzas y la justicia, donde las decisiones tienen un impacto directo en la vida de las personas.

2. **Cumplimiento de normativas:** Cada vez más regulaciones exigen que los modelos de IA sean explicables, como el [GDPR en Europa](https://gdpr-info.eu/), que establece el "derecho a explicación" para decisiones automatizadas.

> **Recursos:** Si quieres saber más sobre el GDPR, te recomiendo explorar este [GPT personalizado](https://chatgpt.com/g/g-JvCAAqPCj-gdpr-expert), está muy bien construido.

3. **Impacto social y ético:** Un modelo opaco puede amplificar sesgos y tomar decisiones injustas. Por ejemplo, si un modelo de contratación favorece a un grupo sobre otro sin una justificación clara, estaríamos perpetuando desigualdades.

En resumen, la interpretabilidad no es solo un *"nice to have"*, es una necesidad fundamental en cualquier aplicación de IA que impacte el mundo real.


## ¿Qué vamos a ver?

En este notebook, nos sumergiremos en el fascinante mundo de la interpretabilidad y exploraremos cómo podemos explicar y justificar las decisiones de dos tipos de modelos muy diferentes: un **árbol de decisión** y una **red neuronal**.

Al final de este notebook, habrás aprendido:

- **Qué es la interpretabilidad y por qué importa**, con ejemplos del mundo real.  
- **Cómo entrenar y visualizar un árbol de decisión,** uno de los modelos más intuitivos.  
- **Cómo trabajar con redes neuronales y explorar su interpretabilidad,** utilizando herramientas como SHAP.  
- **La comparación entre ambos enfoques,** destacando los pros y los contras de cada uno.

La meta no es solo entender qué modelo funciona mejor, sino también aprender a tomar decisiones informadas sobre cuándo priorizar la interpretabilidad frente a la precisión.

**¿Listo para desentrañar las decisiones de tus modelos y convertirte en un maestro de la interpretabilidad?**

**¡Vamos allá! 🚀**

# Índice de Contenidos

```
1. Fundamentos de la Interpretabilidad
   1.1. Componentes clave de la interpretabilidad  
       1.1.1. Comprensibilidad  
       1.1.2. Trazabilidad  
       1.1.3. Generalización y robustez  
   1.2. Dimensiones de la interpretabilidad  
       1.2.1. Modelos intrínsecamente interpretables  
       1.2.2. Interpretabilidad post-hoc  
   1.3. Tradeoff: Interpretabilidad vs. Complejidad  
2. Teoría detrás de los modelos interpretables  
   2.1. Árboles de decisión: Intuición y ventajas  
   2.2. Redes neuronales: El reto de interpretar "cajas negras"  
3. Importancia de la trazabilidad en aplicaciones críticas  
   3.1. Casos de uso: Medicina, Finanzas y Justicia  
   3.2. Problemas por falta de interpretabilidad  
4. Ejercicio práctico: Comparación entre modelos  
   4.1. Dataset: Breast Cancer Wisconsin  
   4.2. Entrenamiento de un Árbol de Decisión  
       4.2.1. Visualización del árbol y análisis de decisiones  
   4.3. Entrenamiento de una Red Neuronal  
       4.3.1. Interpretación mediante SHAP  
5. Comparativa entre Árboles de Decisión y Redes Neuronales  
   5.1. Precisión vs. Interpretabilidad  
   5.2. Ventajas y limitaciones de cada enfoque  
6. Conclusión  
   6.1. Reflexión sobre el tradeoff entre interpretabilidad y precisión  
   6.2. El futuro de la interpretabilidad en modelos avanzados  
```

In [ ]:
import os
import sys

# Configuraciones
# Detectar si estamos en Colab
in_colab = 'google.colab' in str(get_ipython())

if in_colab:
    # Descargar el archivo visualizations.py desde el repositorio de GitHub
    !mkdir -p /content/utils # Creamos una carpeta utils para que coincida con la estructura del repositorio
    !wget -O utils/visualizations.py "https://raw.githubusercontent.com/ManuelEspejo/Machine-Learning-Bases/main/utils/visualizations.py"
    data_dir = '/content/data' # Ruta de los datos
else:
    # Agregar el directorio raíz al path de Python (Para ejecutar en local)
    notebook_dir = os.path.dirname(os.path.abspath('__file__'))
    project_dir = os.path.dirname(notebook_dir)
    sys.path.append(project_dir)
    data_dir = '../data/raw'

# Importar las librerías necesarias
